In [6]:
import sys
import spikeinterface as si
import matplotlib.pyplot as plt
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.widgets as sw
import spikeinterface.qualitymetrics as sqm
import json
import probeinterface

from probeinterface import Probe, ProbeGroup
from probeinterface.plotting import plot_probe, plot_probegroup
from probeinterface import generate_dummy_probe, generate_linear_probe
from probeinterface import write_probeinterface, read_probeinterface
from probeinterface import write_prb, read_prb


In [4]:
file_path = "/media/ubuntu/sda/data/mouse15/ns4/mouse15_052622_natural_image_001.ns4"
probe_position = "/media/ubuntu/sda/data/probe.json"
output_folder = "/media/ubuntu/sda/data/sort_output/mouse15/052622"

global_job_kwargs = dict(n_jobs=10, chunk_duration="1s")
si.set_global_job_kwargs(**global_job_kwargs)

In [ ]:
recording_raw = se.read_blackrock(file_path=file_path)
recording_raw = recording_raw.select_segments([0])

In [11]:
recording_recorded = recording_raw.remove_channels(["98", '31', '32'])
recording_stimulated = recording_raw.channel_slice(['98'])

probe_30channel = read_probeinterface(probe_position)
recording_recorded = recording_recorded.set_probegroup(probe_30channel)

recording_cmr = recording_recorded
recording_f = spre.bandpass_filter(recording_recorded, freq_min=300, freq_max=3000)
print(recording_f)
recording_cmr = spre.common_reference(recording_f, reference="global", operator="median")
print(recording_cmr)

# this computes and saves the recording after applying the preprocessing chain
recording_preprocessed = recording_cmr.save(format="binary")
print(recording_preprocessed)

sorting_kilosort4 = ss.run_sorter(sorter_name="kilosort4", recording=recording_preprocessed, output_folder=output_folder + "/kilosort4")
analyzer_kilosort4 = si.create_sorting_analyzer(sorting=sorting_kilosort4, recording=recording_preprocessed, format='binary_folder', folder=output_folder + '/analyzer_kilosort4_binary')

extensions_to_compute = [
    "random_spikes",
    "waveforms",
    "noise_levels",
    "templates",
    "spike_amplitudes",
    "unit_locations",
    "spike_locations",
    "correlograms",
    "template_similarity"
]

extension_params = {
    "unit_locations": {"method": "center_of_mass"},
    "spike_locations": {"ms_before": 0.1},
    "correlograms": {"bin_ms": 0.1},
    "template_similarity": {"method": "cosine_similarity"}
}

analyzer_kilosort4.compute(extensions_to_compute, extension_params=extension_params)

qm_params = sqm.get_default_qm_params()
analyzer_kilosort4.compute("quality_metrics", qm_params)

BandpassFilterRecording: 30 channels - 10000.0Hz - 1 segments - 24,756,849 samples 
                         2,475.68s (41.26 minutes) - int16 dtype - 1.38 GiB
CommonReferenceRecording: 30 channels - 10000.0Hz - 1 segments - 24,756,849 samples 
                          2,475.68s (41.26 minutes) - int16 dtype - 1.38 GiB
Use cache_folder=/tmp/spikeinterface_cache/tmp__np0kvk/1MV9SF0H
write_binary_recording 
n_jobs=10 - samples_per_chunk=10,000 - chunk_memory=585.94 KiB - total_memory=5.72 MiB - chunk_duration=1.00s


write_binary_recording: 100%|██████████| 2476/2476 [00:02<00:00, 846.00it/s]
/tmp/ipykernel_3175737/4172158192.py:17: DeprecationWarning: `output_folder` is deprecated and will be removed in version 0.103.0 Please use folder instead
  sorting_kilosort4 = ss.run_sorter(sorter_name="kilosort4", recording=recording_preprocessed, output_folder=output_folder + "/kilosort4")


BinaryFolderRecording: 30 channels - 10000.0Hz - 1 segments - 24,756,849 samples 
                       2,475.68s (41.26 minutes) - int16 dtype - 1.38 GiB


estimate_sparsity: 100%|██████████| 2476/2476 [00:00<00:00, 9414.54it/s]
/home/ubuntu/.conda/envs/spike_sorting_jct/lib/python3.10/site-packages/spikeinterface/core/basesorting.py:264: UserWarning: The registered recording will not be persistent on disk, but only available in memory
  warnings.warn("The registered recording will not be persistent on disk, but only available in memory")
Compute : spike_amplitudes + spike_locations: 100%|██████████| 2476/2476 [00:01<00:00, 1866.70it/s]

libgomp: Invalid value for environment variable OMP_NUM_THREADS: KMeans is known to have a memory leak on Windows with MKL
/home/ubuntu/.conda/envs/spike_sorting_jct/lib/python3.10/site-packages/spikeinterface/qualitymetrics/misc_metrics.py:908: UserWarning: Some units have too few spikes : amplitude_cutoff is set to NaN
  warnings.warn(f"Some units have too few spikes : amplitude_cutoff is set to NaN")
/home/ubuntu/.conda/envs/spike_sorting_jct/lib/python3.10/site-packages/numpy/core/_methods.py:206: Run